# Desmascarando Robôs
### CRISP-DM Cycle 1
---

Imagine um mercado online, um palco digital onde diversos leilões se desenrolam a cada segundo. Neste ambiente, participantes do mundo inteiram lançam seus lances em busca de objetos desejados, desde joias até equipamentos tecnológicos. No entanto, nem todos os jogadores neste campo são humanos; alguns são robôs programados para manipular os resultados dos leilões.

Seu desafio é se aprofundar nesses dados, explorar as camadas de atividade nos leilões e conseguir construir um modelo que saiba muito bem diferenciar humanos de robôs.

> Disclaimer: This is a fictional bussiness cas

## 0. PREPARATION

### 0.1 Settings

In [1]:
# Settings imports
import os
import sys
import pandas as pd
import pyarrow as pa
import pyarrow.feather as feather
from dotenv import load_dotenv

# Load .env file
env_path = "../.env"
load_dotenv(dotenv_path=env_path)

# Seed
seed = int(os.getenv("SEED"))

# Add path
path = os.getenv("HOMEPATH")

# Add path to sys.path
sys.path.append(path)

In [2]:
# import helper.functions.descriptive_statistics as ds

# from helper.classes.FeatureSelection import FeatureSelection
# from helper.classes.MLPipeline import MLPipeline, MLPipelineCV

### 0.2 Data

**Train e Test**

- **id_participante**: Identificador único do participante
- **conta_pagamento**: Conta de pagamento associada ao participante (com o valor ocultado)
- **endereco**: Endereço postal do participante
- **resultado**: A variável alvo que identifica se o participante é um robô ou um humano. (Robô = 1 e Humano = 0). (*target*)

- **Robôs Confirmados**: Participantes com provas claras de atividades fraudulentas, resultando em banimento da plataforma. São rotulados como robôs no conjunto de dados (resultado = 1).

- **Robôs Suspeitos**: Participantes com atividades atípicas ou estatísticas que superam a média, mas sem provas definitivas de fraude. A classificação deles como robôs é incerta.

**Lances**

- **id_lance**: Identificador único do lance
- **id_participante**: Identificador único do participante
- **leilao**: Identificador único do leilão
- **mercadoria**: A categoria da mercadoria leiloada
- **dispositivo**: O dispositivo utilizado pelo visitante
- **tempo**: O tempo que o lance foi feito
- **pais**: O país que o IP pertence
- **ip**: O IP do participante
- **url**: A URL de onde o participante foi referido

In [3]:
from helper.classes.Queries import DuckQueries

qb = DuckQueries()
conn = qb.get_connection(path + "/data/interim/data.db")

# Train
query_train = qb.select("*").from_table("train").build()
train = conn.execute(query_train).df()

# Test
query_test = qb.select("*").from_table("test").build()
test = conn.execute(query_test).df()

# Lances
query_lances = qb.select("*").from_table("lances").build()
df_lances = conn.execute(query_lances).df()

# Concatenate train and test
df = pd.concat([train, test], ignore_index=True)

# Join train with lances in id_participante
df_train = train.merge(df_lances, on="id_participante", how="left")
df_train = pa.Table.from_pandas(df_train)

# Join test with lances in id_participante
df_test = test.merge(df_lances, on="id_participante", how="left")
df_test = pa.Table.from_pandas(df_test)

# Join df with lances in id_participante
data = df.merge(df_lances, on="id_participante", how="left")
data = pa.Table.from_pandas(data)

# Save data as feather
feather.write_feather(df_train, path + "/data/interim/df_train.feather")
feather.write_feather(df_test, path + "/data/interim/df_test.feather")
feather.write_feather(data, path + "/data/interim/data.feather")

## 1. DATA UNDERSTANDING

### 1.1 Loading Data

In [4]:
data = pd.read_feather(path + "/data/interim/data.feather")

### 1.2 Data Understanding

#### 1.2.1 Shape

In [5]:

print(f"O dataset possui {data.shape[0]} linhas e {data.shape[1]} colunas\n")


O dataset possui 3071253 linhas e 12 colunas



#### 1.2.2 Data Types

In [6]:
data.dtypes

id_participante     object
conta_pagamento     object
endereco            object
resultado          float64
id_lance           float64
leilao              object
mercadoria          object
dispositivo         object
tempo              float64
pais                object
ip                  object
url                 object
dtype: object

#### 1.2.3 NA

In [7]:
# check for missing values
data.isnull().sum()

id_participante         0
conta_pagamento         0
endereco                0
resultado          536912
id_lance               29
leilao                 29
mercadoria             29
dispositivo            29
tempo                  29
pais                 2730
ip                     29
url                    29
dtype: int64

In [8]:
df.isnull().sum()

id_participante      0
conta_pagamento      0
endereco             0
resultado          604
dtype: int64

In [9]:
df_train.isnull().sum()

AttributeError: 'pyarrow.lib.Table' object has no attribute 'isnull'

In [ ]:
df_test.isnull().sum()

In [10]:
train.isnull().sum()

id_participante    0
conta_pagamento    0
endereco           0
resultado          0
dtype: int64

In [11]:
test.isnull().sum()

id_participante    0
conta_pagamento    0
endereco           0
dtype: int64

In [13]:
df_lances.isnull().sum()

id_lance              0
id_participante       0
leilao                0
mercadoria            0
dispositivo           0
tempo                 0
pais               8859
ip                    0
url                   0
dtype: int64